In [4]:
import os 

In [5]:
%pwd

'c:\\Users\\Asus\\Desktop\\kidney_Disease_Classifier\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\Asus\\Desktop\\kidney_Disease_Classifier'

In [8]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sharmantima1010"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b6e96687cb64b794bcb2f2b3b6a1f3651e45009f"

In [9]:
import tensorflow as tf

In [10]:
model= tf.keras.models.load_model("artifacts/training/model.keras")

In [18]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int
    

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )


    

        return eval_config

In [14]:
pip install mlflow==2.17.0


  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.20.0 requires numpy>=1.26.0, but you have numpy 1.25.0 which is incompatible.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 3.11.3 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.5 which is incompatible.
tensorflow-intel 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.20.0 which is incompatible.


In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):
        
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self._valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        model=tf.keras.models.load_model(path)
        return model
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self._valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(
            path=Path("scores.json"),
            data=scores
        )
    

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="KidneyDiseaseClassifierModel"
                )
            else:
                mlflow.keras.log_model(
                    self.model,
                    "model"
                )
            
        
    

In [21]:
try:
    config= ConfigurationManager()
    eval_config=config.get_evaluation_config()
    
    evaluation= Evaluation(eval_config)
    evaluation.evaluation()
    
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-10-28 21:25:07,509: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-28 21:25:07,513: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-28 21:25:07,515: INFO: common: created directory at: artifacts_root]
Found 3732 images belonging to 4 classes.


c:\Users\Asus\anaconda3\envs\kidney\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


234/234 ━━━━━━━━━━━━━━━━━━━━ 163s 687ms/step - accuracy: 0.8950 - auc: 0.9787 - loss: 0.6790 - precision: 0.9013 - recall: 0.8877
[2025-10-28 21:27:51,397: INFO: common: json file saved at: scores.json]


2025/10/28 21:27:54 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/10/28 21:28:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Asus\AppData\Local\Temp\tmpzji3idpp\model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/28 21:28:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-10-28 21:37:22,697: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /sharmantima1010/Kidney_Disease_Classifier.mlflow/api/2.0/mlflow-artifacts/artifacts/3ee4eb50c9834ad08dbecb12451458e3/89d5a5ee0fee4a0499965a25e6f6739e/artifacts/model/data/model.keras]
[2025-10-28 21:47:24,421: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /sharmantima1010/Kidney_Disease_Classifier.mlflow/api/2.0/mlflow-artifacts/artifacts/3ee4eb50c9834ad08dbecb12451458e3/89d5a5ee0fee4a0499965a25e6f6739e/artifacts/model/data/model.keras]
[2025-10-28 21:51:01,812: WARNING: connectionpool: Retrying (Retry(total=2, connect=5, read=2, redirect=5, status=5)) after connection broken by 'TimeoutError('The write operation timed out')': /sharmantima1010/Kidney_Disease_Classifier.mlflo

Successfully registered model 'KidneyDiseaseClassifierModel'.
2025/10/28 21:56:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: KidneyDiseaseClassifierModel, version 1
Created version '1' of model 'KidneyDiseaseClassifierModel'.
2025/10/28 21:56:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run clean-mouse-690 at: https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow/#/experiments/0/runs/89d5a5ee0fee4a0499965a25e6f6739e.
2025/10/28 21:56:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sharmantima1010/Kidney_Disease_Classifier.mlflow/#/experiments/0.


In [22]:
import dagshub
dagshub.init(repo_owner='sharmantima1010', repo_name='Kidney_Disease_Classifier', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

ModuleNotFoundError: No module named 'dagshub'